#### CMSC 170: Laboratory Exercise 2
##### Missionaries and Cannibals

### Introduction

#### Requirements
* pygame
* sys
* time
* collections
* typing
* copy

#### Problem Analysis Notes
* ALL missionaries and cannibals must be transfered from the left to the ride side of the river
* Ensure that missionaries are never outnumbered by cannibals on either side
* Boat has a capacity constraint of 2
* Initial state is (3,3,1): 3 missionaries, 3 cannibals, and the boat on the left side
* The goal state is (0,0,0): everyone including the boat on the right side
* Valid actions consist of moving 1 or 2 people across through the boat AND someone must always be in the boat

Therefore,
* Valid states are:
    * Missionaries >= Cannibals on each side unless missionaries = 0
    * Total people conservation (6)
    * Boat position consistency

I. State Representation Class

In [22]:
class GameState:

    def __init__(self, missionaries_left: int, cannibals_left: int, boat_left: bool):
        
        self.missionaries_left = missionaries_left
        self.cannibals_left = cannibals_left
        self.boat_left = boat_left

        # calculation of entities on the right side (total conservation)
        self.missionaries_right = 3 - missionaries_left
        self.cannibals_right = 3 - cannibals_left

    def __eq__(self, other) -> bool:

        # for state comparison for duplicate detection for search algorithms
        if not isinstance(other, GameState):
            return False
        return (self.missionaries_left == other.missionaries_left and
                self.cannibals_left == other.cannibals_left and
                self.boat_left == other.boat_left)

    def __hash__(self) -> int:

        # use as dictionary key and in sets
        return hash((self.missionaries_left, self.cannibals_left, self.boat_left))

    def __str__(self) -> str:

        # print boat emoji on the left if the boat is on the left, and vice versa
        if self.boat_left:
            return f"Left: M = {self.missionaries_left}, C = {self.cannibals_left}, 🛶 | " \
                   f"Right: M = {self.missionaries_right}, C = {self.cannibals_right}"
        else:
            return f"Left: M = {self.missionaries_left}, C = {self.cannibals_left} | " \
                   f"Right: M = {self.missionaries_right}, C = {self.cannibals_right}, 🛶"


### II. Logic and Validation

In [37]:
from typing import List, Tuple, Optional
import copy

class MissionariesCannibals:

    def __init__(self):

        # initialize the starting state
        self.initial_state = GameState(3, 3, True)
        self.current_state = GameState(3, 3, True)
        self.goal_state = GameState(0, 0, False)
        self.move_history: List[GameState] = [copy.deepcopy(self.current_state)]

    def is_valid(self, state: GameState) -> bool:

        # checking based on the analysis notes aforementioned
        # 1. non-negativity
        # 2. total conservation
        # 3. missionaries must not be outnumbered if missionaries != 0

        # function returns a boolean whether the state is valid or not

        # rule 1
        if (state. missionaries_left < 0 or state.cannibals_left < 0 or 
            state.missionaries_right < 0 or state.cannibals_right < 0):
            
            return False

        # rule 2
        if (state.missionaries_left + state.missionaries_right != 3 or
            state.cannibals_left + state.cannibals_right != 3):

            return False

        # rule 3
        # left
        if (state.missionaries_left > 0 and
            state.missionaries_left < state.cannibals_left):

            return False

        if (state.missionaries_right > 0 and
            state.missionaries_right < state.cannibals_right):

            return False
            
        return True

    # check if goal state is achieved
    def is_goal(self, state: GameState) -> bool:

        return state == self.goal_state

    # generate possible moves from current state
    def possible_moves(self, state: GameState) -> List[Tuple[int, int]]:

        # move representation: (missionaries, cannibals) to put in boat

        moves = []

        possible_loads = [
            (1, 0), # 1 missionary
            (0, 1), # 1 cannibal
            (2, 0), # 2 missionaries
            (0, 2), # 2 cannibals
            (1, 1) # 1 missionary and 1 cannibal
        ]

        for m_move, c_move in possible_loads:
            
            # check if enough number of missionary and cannibal are enough on the current side
            if state.boat_left:
                if (state.missionaries_left >= m_move and
                    state.cannibals_left >= c_move):
                    moves.append((m_move, c_move))

            else:
                if (state.missionaries_right >= m_move and
                    state.cannibals_right >= c_move):
                    moves.append((m_move, c_move))
        return moves

    # apply move to a state then return the resulting state, if valid
    def apply_move(self, state: GameState, move: Tuple[int, int]) -> Optional[GameState]:

        m_move, c_move = move

        if state.boat_left:
            # from left to right
            new_state = GameState(
                state.missionaries_left - m_move,
                state.cannibals_left - c_move,
                False
            )
        else:
            # from right to left
            new_state = GameState(
                state.missionaries_left + m_move,
                state.cannibals_left + c_move,
                True
            )

        if self.is_valid(new_state):
            return new_state
        return None

    def make_move(self, missionaries: int, cannibals: int) -> bool:
        
        # function for interactivity, returns bool True if successful
        move = (missionaries, cannibals)
        new_state = self.apply_move(self.current_state, move)

        if new_state:
            self.current_state = new_state
            self.move_history.append(copy.deepcopy(new_state))
            return True
        return False

    def reset(self):
        self.current_state = GameState(3, 3, True)
        self.move_history = [copy.deepcopy(self.current_state)]

### III. Search Algorithms

In [41]:
from collections import deque
from typing import Set

class SearchAlgorithms:
    # implementation of BFS and DFS for solving the problem

    @staticmethod
    def bfs(game: MissionariesCannibals) -> Optional[List[GameState]]:
        # guaranteed shortest path
        # queue data structure

        queue = deque([(game.initial_state, [game.initial_state])])
        visited: Set[GameState] = {game.initial_state}

        nodes_explored = 0

        while queue:
            current_state, path = queue.popleft()
            nodes_explored += 1

            # check if goal is achieved
            if game.is_goal(current_state):
                print(f"solution found, explored: {nodes_explored} nodes.")

                choice = input("do you want to print entire solution path (y/n)? ").strip().lower()
                if choice == "y":
                    for step_num, state in enumerate(path):
                        print(f"step {step_num}: {state}")
                              
                return path

            # generate and explore all possible next states
            for move in game.possible_moves(current_state):
                next_state = game.apply_move(current_state, move)

                if next_state and next_state not in visited:
                    visited.add(next_state)
                    new_path = path + [next_state]
                    queue.append((next_state, new_path))

        print("no solution found")
        return None

    @staticmethod
    def dfs(game: MissionariesCannibals, max_depth: int = 20) -> Optional[List[GameState]]:
        # dfs goes into one path before backtracking
        # stack data struct

        def dfs_recursive(state: GameState, path: List[GameState],
                          visited: Set[GameState], depth: int) -> Optional[List[GameState]]:
            if depth > max_depth:
                return None

            if game.is_goal(state):
                return path

            # explore all possible moves
            for move in game.possible_moves(state):
                next_state = game.apply_move(state, move)

                if (next_state and next_state not in visited and
                    next_state not in path):
                    # avoiding cycles by checking visited moves

                    visited.add(next_state)
                    result = dfs_recursive(next_state, path + [next_state], visited, depth + 1)

                    if result:
                        return result

                    visited.remove(next_state)

            return None

        visited: Set[GameState] = {game.initial_state}
        solution = dfs_recursive(game.initial_state, [game.initial_state], visited, 0)

        if solution:
            print(f"solution found in {len(solution) - 1} moves")
            
            choice = input("do you want to print entire solution path (y/n)? ").strip().lower()
            if choice == "y":
                for step_num, state in enumerate(solution):
                    print(f"step {step_num}: {state}")
                          
            return solution
        else:
            print("no solution found within depth limit")

In [ ]:
game = MissionariesCannibals()

print("=== bfs ===")
bfs_solution = SearchAlgorithms.bfs(game)

print("=== dfs ===")
dfs_solution = SearchAlgorithms.dfs(game, max_depth = 20)

solution found, explored: 15 nodes.


do you want to print entire solution path (y/n)?  n


solution found in 11 moves


### Command Line Interface Game

In [ ]:
class CommandLineInterface:
    def __init__(self):
        self.game = MissionariesCannibalsGame()